## United States Economic Forecast Project- Oussama Ennaciri
## Data Preparation
We will retrieve our economic indicators data from the `raw` schema, clean the missing values, and load the data into the newly created `cleaned` schema.

In [36]:
import seaborn as sns
import pandas as pd
import numpy as np

In [37]:
from sqlalchemy import create_engine

In [38]:
host = r'127.0.0.1'
db = r'MSDS610' 
user = r'postgres'
pw = r'password' 
port = r'5432' 

In [39]:
db_conn = create_engine("postgresql://{}:{}@{}:{}/{}".format(user, pw, host, port, db))

## Cleaning the Monthly Data

In [40]:
table_name = r'monthly_data'
schema = r'raw' 

monthly_raw = pd.read_sql_table(table_name, db_conn, schema)

In [41]:
monthly_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 781 entries, 0 to 780
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   observation_date  781 non-null    object 
 1   PCE               780 non-null    float64
 2   HOUST             780 non-null    float64
 3   UNRATE            780 non-null    float64
 4   FEDFUNDS          781 non-null    float64
 5   GS10              781 non-null    float64
 6   CPIAUCSL          780 non-null    float64
dtypes: float64(6), object(1)
memory usage: 42.8+ KB


In [42]:
monthly_raw.tail(10)

,observation_date,PCE,HOUST,UNRATE,FEDFUNDS,GS10,CPIAUCSL
771,2024-04-01,19603.3,1377.0,3.9,5.33,4.54,313.207
772,2024-05-01,19697.3,1315.0,4.0,5.33,4.48,313.225
773,2024-06-01,19747.5,1329.0,4.1,5.33,4.31,313.049
774,2024-07-01,19866.3,1262.0,4.2,5.33,4.25,313.534
775,2024-08-01,19905.0,1379.0,4.2,5.33,3.87,314.121
776,2024-09-01,20044.1,1355.0,4.1,5.13,3.72,314.686
777,2024-10-01,20134.5,1344.0,4.1,4.83,4.10,315.454
778,2024-11-01,20253.6,1294.0,4.2,4.64,4.36,316.441
779,2024-12-01,20387.2,1499.0,4.1,4.48,4.39,317.685
780,2025-01-01,NaN,NaN,NaN,4.33,4.63,NaN


### Checking for null values in the monthly dataset

In [43]:
monthly_raw.isnull().sum()

observation_date    0
PCE                 1
HOUST               1
UNRATE              1
FEDFUNDS            0
GS10                0
CPIAUCSL            1
dtype: int64

In [44]:
monthly_clean = monthly_raw.copy()

### Dropping rows with null values in the monthly dataset

In [45]:
monthly_clean.dropna(subset=['PCE', 'HOUST', 'UNRATE', 'FEDFUNDS', 'GS10', 'CPIAUCSL' ], inplace=True)
monthly_clean.isnull().sum()

observation_date    0
PCE                 0
HOUST               0
UNRATE              0
FEDFUNDS            0
GS10                0
CPIAUCSL            0
dtype: int64

### Converting date columns to datetime format

In [46]:
monthly_clean['observation_date'] = pd.to_datetime(monthly_clean['observation_date'])

### Checking for duplicates in the monthly dataset

In [64]:
monthly_raw[monthly_raw.duplicated(subset=['observation_date'])]

,observation_date,PCE,HOUST,UNRATE,FEDFUNDS,GS10,CPIAUCSL


### Adding a recession indicator (USREC) to mark periods of  recession

In [48]:
usrec = pd.read_csv('economic_indicators/USREC.csv')
usrec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 781 entries, 0 to 780
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   observation_date  781 non-null    object
 1   USREC             781 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 12.3+ KB


In [49]:
usrec['observation_date'] = pd.to_datetime(usrec['observation_date'])

In [50]:
usrec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 781 entries, 0 to 780
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   observation_date  781 non-null    datetime64[ns]
 1   USREC             781 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 12.3 KB


In [51]:
monthly_clean = pd.merge(monthly_clean, usrec, on='observation_date')
monthly_clean

,observation_date,PCE,HOUST,UNRATE,FEDFUNDS,GS10,CPIAUCSL,USREC
0,1960-01-01,323.6,1460.0,5.2,3.99,4.72,29.370,0
1,1960-02-01,325.3,1503.0,4.8,3.97,4.49,29.410,0
2,1960-03-01,330.2,1109.0,5.4,3.84,4.25,29.410,0
3,1960-04-01,336.5,1289.0,5.2,3.92,4.28,29.540,0
4,1960-05-01,330.0,1271.0,5.1,3.85,4.35,29.570,1
...,...,...,...,...,...,...,...,...
775,2024-08-01,19905.0,1379.0,4.2,5.33,3.87,314.121,0
776,2024-09-01,20044.1,1355.0,4.1,5.13,3.72,314.686,0
777,2024-10-01,20134.5,1344.0,4.1,4.83,4.10,315.454,0
778,2024-11-01,20253.6,1294.0,4.2,4.64,4.36,316.441,0


### Saving the cleaned data to the cleaned schema

In [52]:
table_name1 = r'monthly_clean'
schema = r'cleaned'

monthly_clean.to_sql(table_name, con=db_conn, if_exists='replace', index=False, schema=schema, chunksize=1000, method='multi')

780

### Cleaning the Quarterly Data

In [54]:
table_name = r'quarterly_data'
schema = r'raw' 

quarterly_raw = pd.read_sql_table(table_name, db_conn, schema)

In [55]:
quarterly_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   observation_date  260 non-null    object 
 1   GDP               260 non-null    float64
 2   GDPC1             260 non-null    float64
 3   A053RC1Q027SBEA   259 non-null    float64
 4   IMPGSC1           260 non-null    float64
 5   EXPGSC1           260 non-null    float64
 6   GCEC1             260 non-null    float64
dtypes: float64(6), object(1)
memory usage: 14.3+ KB


### Checking for null values in the quarterly dataset

In [56]:
quarterly_raw.isnull().sum()

observation_date    0
GDP                 0
GDPC1               0
A053RC1Q027SBEA     1
IMPGSC1             0
EXPGSC1             0
GCEC1               0
dtype: int64

In [57]:
quarterly_clean = quarterly_raw.copy()

### Dropping rows with null values in the quarterly dataset

In [58]:
quarterly_clean.dropna(subset=['GDP', 'GDPC1', 'A053RC1Q027SBEA', 'IMPGSC1', 'EXPGSC1', 'GCEC1' ], inplace=True)
quarterly_clean.isnull().sum()

observation_date    0
GDP                 0
GDPC1               0
A053RC1Q027SBEA     0
IMPGSC1             0
EXPGSC1             0
GCEC1               0
dtype: int64

### Converting date columns to datetime format

In [59]:
quarterly_clean['observation_date'] = pd.to_datetime(quarterly_clean['observation_date'])

### Checking for duplicates in the quarterly dataset

In [63]:
quarterly_raw[monthly_raw.duplicated(subset=['observation_date'])]

/var/folders/8p/h9s_4hcj6hnfgmq4b6yz667r0000gn/T/ipykernel_7941/2888167018.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  quarterly_raw[monthly_raw.duplicated(subset=['observation_date'])]


,observation_date,GDP,GDPC1,A053RC1Q027SBEA,IMPGSC1,EXPGSC1,GCEC1


### Creating an export-to-import ratio column to enhance economic analysis

In [61]:
quarterly_clean["export_to_import_ratio"] = quarterly_clean["EXPGSC1"] / quarterly_clean["IMPGSC1"]
quarterly_clean

,observation_date,GDP,GDPC1,A053RC1Q027SBEA,IMPGSC1,EXPGSC1,GCEC1,export_to_import_ratio
0,1960-01-01,542.648,3517.181,58.778,123.608,104.686,1137.363,0.846919
1,1960-04-01,541.080,3498.246,54.428,124.527,107.689,1150.170,0.864784
2,1960-07-01,545.604,3515.385,52.155,120.857,110.036,1178.324,0.910464
3,1960-10-01,540.197,3470.278,50.050,115.156,110.622,1186.511,0.960627
4,1961-01-01,545.018,3493.703,49.179,114.599,109.861,1204.136,0.958656
...,...,...,...,...,...,...,...,...
254,2023-07-01,27967.697,22780.933,3748.928,3460.391,2521.467,3836.304,0.728665
255,2023-10-01,28296.967,22960.600,3887.693,3496.290,2559.591,3870.720,0.732088
256,2024-01-01,28624.069,23053.545,3962.994,3548.749,2571.763,3887.718,0.724696
257,2024-04-01,29016.714,23223.906,4088.703,3614.047,2578.386,3917.049,0.713435


### Saving the cleaned data to the cleaned schema

In [62]:
table_name2 = r'quarterly_clean'
schema = r'cleaned'

quarterly_clean.to_sql(table_name, con=db_conn, if_exists='replace', index=False, schema=schema, chunksize=1000, method='multi')

259